In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：
bounds = [(0,10),(0,10),(0,10)]
X_optimal = [5,5,5]
y_optimal = -1

def G12(X):
    y = -(100-(X[0]-5)**2-(X[1]-5)**2-(X[2]-5)**2)/100
    g = 0
    for p in range(1,10):
        for q in range(1,10):
            for r in range(1,10):
                if (X[0]-p)**2+(X[1]-q)**2+(X[2]-r)**2-0.0625 < g:
                    g = (X[0]-p)**2+(X[1]-q)**2+(X[2]-r)**2-0.0625
    return y,g

In [4]:
### 适应度函数
def fitnessG12_num(X, epsion):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = -0.25
    fmin = -1
    count = np.sum(np.array(G12(X))[1:2]>0)
    
    return -(G12(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG12_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = -0.25
    fmin = -1
    vmax = 243
    e = 0.1
    n = 1
    
    res = G12(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:2]>0)
    
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness    


In [5]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-100000000.0)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-100000000.0)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG12_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
            w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG12_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [6]:
PSO = PSO_fit(pN=400, dim=3, max_iter=100, bounds=bounds)

In [7]:
PSO.init_Population()

In [8]:
PSO.iterator()

[0.90508344 0.91628498 0.91662715 0.82881008 0.87266798 0.92185786
 0.88481966 0.98392342 0.97481174 0.91707167 0.91167514 0.88138167
 0.96448742 0.87253696 0.94705056 0.91831339 0.89147078 0.96471771
 0.91195579 0.96471771 0.82328676 0.88154827 0.91707167 0.97481174
 0.90508344 0.86464219 0.8372672  0.91831339 0.97481174 0.96252866
 0.88456405 0.91403794 0.91859723 0.98050121 0.93247117 0.9804436
 0.95085028 0.97461675 0.94542479 0.91662715 0.88788197 0.90508344
 0.73102191 0.90162494 0.88600195 0.88609178 0.95495036 0.91403794
 0.95428772 0.88109542 0.91866971 0.95428772 0.90127699 0.98050121
 0.88600195 0.91866971 0.91147907 0.96714841 0.98050121 0.84906064
 0.96714841 0.9314873  0.86216285 0.88367353 0.93796684 0.85997343
 0.82881008 0.95085028 0.91866971 0.92391184 0.91859723 0.89493437
 0.91662715 0.99272368 0.76101886 0.9804436  0.9100277  0.98392342
 0.95495036 0.91469894 0.89493437 0.90922476 0.97827177 0.88154827
 0.82130393 0.92185786 0.93796684 0.82787592 0.93247117 0.98050

[0.98896644 0.96836424 0.96803769 0.97481174 0.99272368 0.98915389
 0.96005935 0.98392342 0.97481174 0.94564127 0.96471771 0.98008152
 0.97827177 0.99272368 0.96855585 0.91831339 0.93247117 0.96535022
 0.98072037 0.96471771 0.98008152 0.93703376 0.95151537 0.97481174
 0.90508344 0.98946525 0.96836424 0.96535022 0.9901004  0.98072037
 0.98008152 0.98072037 0.93012214 0.98050121 0.93554236 0.9901004
 0.96075594 0.98896644 0.97528958 0.96577276 0.97143971 0.90813564
 0.91959506 0.98008152 0.92362669 0.97234742 0.97827177 0.99599478
 0.95428772 0.89676273 0.96490416 0.96836424 0.94455392 0.98050121
 0.98072037 0.93708354 0.96803769 0.9901004  0.98050121 0.94560605
 0.97827177 0.98392342 0.99272368 0.94542479 0.95428772 0.92664268
 0.94090898 0.95085028 0.98946525 0.92391184 0.99339804 0.92185786
 0.91662715 0.99272368 0.97827177 0.9804436  0.95371714 0.9905898
 0.96075594 0.96252866 0.89493437 0.96803769 0.97827177 0.93640914
 0.9901004  0.93367855 0.98031034 0.96557932 0.93247117 0.980501

[0.98896644 0.98437317 0.97186511 0.97481174 0.99272368 0.98915389
 0.98915389 0.98392342 0.97827177 0.96075594 0.96471771 0.9905898
 0.97827177 0.99659348 0.96855585 0.93554236 0.97461675 0.97022577
 0.98938853 0.97026976 0.98008152 0.98943704 0.9677914  0.97481174
 0.98943704 0.98946525 0.96836424 0.96535022 0.9901004  0.98072037
 0.99519362 0.98072037 0.97018659 0.98050121 0.95660751 0.9901004
 0.97075999 0.99599478 0.97827177 0.97827177 0.97143971 0.98031034
 0.99519362 0.98008152 0.98050121 0.97234742 0.97827177 0.99599478
 0.98050121 0.98938853 0.97768334 0.97075999 0.98437317 0.98280577
 0.98072037 0.99519362 0.97977581 0.9901004  0.98050121 0.99822325
 0.97827177 0.98392342 0.99711031 0.97505441 0.97241221 0.9901004
 0.97481174 0.97018659 0.98946525 0.9715445  0.99339804 0.95263727
 0.96075594 0.99272368 0.97827177 0.9804436  0.99711031 0.99711031
 0.99036165 0.98590623 0.97461675 0.99822325 0.97827177 0.96641996
 0.9901004  0.96942276 0.98031034 0.97768334 0.98437317 0.9839234

[0.9905898  0.9905898  0.97186511 0.99599478 0.99711031 0.99762162
 0.99822325 0.99353598 0.99599478 0.98347619 0.9715445  0.9905898
 0.9804436  0.99659348 0.96855585 0.98103896 0.99339804 0.99853247
 0.99822325 0.97678913 0.98590623 0.99239784 0.99599478 0.97567744
 0.98943704 0.98946525 0.99979964 0.98903896 0.9901004  0.98072037
 0.99519362 0.98700845 0.97020425 0.98982277 0.98847735 0.99075421
 0.98943704 0.99599478 0.99036165 0.97827177 0.97143971 0.99824958
 0.99519362 0.98773342 0.98773342 0.99853247 0.97827177 0.99599478
 0.99752891 0.98938853 0.98943704 0.9987476  0.99979964 0.98280577
 0.98915389 0.99519362 0.97977581 0.9901004  0.98050121 0.99979964
 0.98059855 0.99762162 0.99711031 0.99824958 0.9769714  0.9901004
 0.98982277 0.98826102 0.9901004  0.9715445  0.99979964 0.96075594
 0.96075594 0.99274819 0.99036165 0.9804436  0.99711031 0.99711031
 0.99853247 0.98590623 0.98813295 0.99822325 0.9987476  0.9664705
 0.9901004  0.99272368 0.9987476  0.98943704 0.9901004  0.9923978

[0.99635689 0.99352499 0.99777486 0.99606295 0.99711031 0.9987476
 0.99822325 0.9995319  0.99752891 0.98700845 0.98269077 0.99239784
 0.99519362 0.99659348 0.99853247 0.99339804 0.99339804 0.99915837
 0.99822325 0.98946525 0.9901004  0.99752891 0.99777486 0.98280577
 0.99771721 0.99632287 0.99979964 0.99038188 0.99635689 0.99854027
 0.99519362 0.99392504 0.97489009 0.98982277 0.98875455 0.99147487
 0.98943704 0.99599478 0.99036165 0.994553   0.99915837 0.99824958
 0.99549074 0.99771721 0.99908536 0.99853247 0.9997782  0.99599478
 0.99752891 0.98938853 0.98943704 0.9987476  0.99979964 0.99036165
 0.98915389 0.99519362 0.99464144 0.9901004  0.9850329  0.99979964
 0.99467069 0.99762162 0.99915837 0.99824958 0.98590623 0.9901004
 0.99392504 0.99353598 0.9901004  0.98896644 0.99979964 0.99632287
 0.9995319  0.99339804 0.99853247 0.99606295 0.99711031 0.99771721
 0.99853247 0.99854027 0.99979964 0.99822325 0.9987476  0.99392504
 0.9901004  0.99272368 0.9987476  0.98943704 0.99979964 0.996062

...

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

(array([[4.99999998, 5.00000007, 5.00000004],
        [4.99999991, 4.99999996, 5.00000001],
        [4.99999994, 5.00000001, 5.00000005],
        ...,
        [5.00000004, 4.99999997, 5.00000004],
        [4.99999997, 4.99999996, 5.00000004],
        [5.00000014, 5.00000008, 5.00000007]]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1